# Treinamento do modelo usando Random Forest no SageMaker

### Importando bibliotecas e variáveis de ambiente 

In [ ]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Carregar variáveis de ambiente a partir de um arquivo .env
load_dotenv()

# Definir a região a ser usada (us-east-1)
region_name = 'us-east-1'

# Carregar variáveis de ambiente para conectar ao banco de dados RDS
database_name = os.getenv('DATABASE_NAME')
table_name = os.getenv('TABLE_NAME')
rds_host = os.getenv('RDS_HOST')
rds_port = os.getenv('RDS_PORT')
rds_user = os.getenv('RDS_USER')
rds_password = os.getenv('RDS_PASSWORD')

# Criar uma sessão boto3
boto3_session = boto3.Session()

# Criar um cliente S3 usando a sessão boto3
s3_client = boto3_session.client('s3')

# Criar um cliente SageMaker usando a sessão boto3
sm_boto3 = boto3_session.client('sagemaker')

# Criar uma sessão SageMaker
sess = sagemaker.Session(boto_session=boto3_session)

# Obter a região da sessão boto3
region = sess.boto_session.region_name

# Definir o nome do bucket S3 a partir da variável de ambiente
bucket = os.getenv('BUCKET_NAME')

# Imprimir o nome do bucket que está sendo usado
print("Usando o bucket " + bucket)


### Importando a database do RDS

In [ ]:
# Criar a string de conexão para o SQLAlchemy
db_url = f'mysql+pymysql://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{database_name}'

# Conectar-se ao banco de dados
engine = create_engine(db_url)

# Consultar o banco de dados e carregar os dados em um DataFrame
query = f'SELECT * FROM {table_name}'
hotel_reservas = pd.read_sql(query, con=engine)
hotel_reservas